In [1]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict, json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib notebook
import gmaps
import os


In [2]:
#url1 ="https://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&zpid=48749425"
#url2 ="http://www.zillow.com/webservice/GetDeepComps.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&zpid=48749425&count=5"
#url3 ="http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA"
url4="http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&state=nj"
#&childtype=county
#url5="http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&state=nj&city=jersey&childtype=neighborhood"

In [3]:
#Request API
#response1 = requests.get(url1).content
#response2 = requests.get(url2).content
#response3 = requests.get(url3).content
response4 = requests.get(url4).content

In [4]:
#convert XML to json

#resp1 = xmltodict.parse(response1)
#resp2 = xmltodict.parse(response2)
#resp3 = xmltodict.parse(response3)
resp4 = xmltodict.parse(response4)

#json.dumps(resp1) 
#json.dumps(resp2)
#json.dumps(resp3)
json.dumps(resp4)
#convert json to Dataframe

#propertydetails_df = pd.DataFrame.from_dict(json_normalize(resp1), orient='columns')
#comps_df = pd.DataFrame.from_dict(json_normalize(resp2), orient='columns')
#searchresults_df = pd.DataFrame.from_dict(json_normalize(resp3), orient='columns')
regionchildren_df = pd.DataFrame.from_dict(json_normalize(resp4), orient='columns')


In [5]:
#propertydetails_df.rename(columns=lambda x: x[55:], inplace=True)
#comps_df.rename(columns=lambda x: x[45:], inplace=True)
#searchresults_df.rename(columns=lambda x: x[30:], inplace=True)
regionchildren_df.rename(columns=lambda x: x[30:], inplace=True)
regionchildren_df.head(5)

,@xsi:schemaLocation,@xmlns:xsi,@xmlns:RegionChildren,request.state,message.text,message.code,response.region.id,response.region.latitude,response.region.longitude,response.subregiontype,response.list.count,response.list.region
0,http://www.zillow.com/static/xsd/RegionChildre...,http://www.w3.org/2001/XMLSchema-instance,http://www.zillow.com/static/xsd/RegionChildre...,nj,Request successfully processed,0,40,40.14296,-74.727272,county,21,"[{'id': '874', 'name': 'Bergen County', 'zinde..."


In [6]:
#extract child(zipcode) which is a dictionary into a dataframe
region_list = pd.DataFrame(regionchildren_df['response.list.region'][0])
#dropna and reset index
region_list.dropna(how="any", inplace=True)
region_list.reset_index(drop=True, inplace=True)

print(region_list.head(5))

     id              name                                   zindex   latitude  \
0   874     Bergen County  {'@currency': 'USD', '#text': '424000'}  40.948055   
1  2802  Middlesex County  {'@currency': 'USD', '#text': '293300'}  40.430013   
2  1106     Hudson County  {'@currency': 'USD', '#text': '321000'}  40.732859   
3  2441   Monmouth County  {'@currency': 'USD', '#text': '375700'}  40.302335   
4   659      Ocean County  {'@currency': 'USD', '#text': '255700'}  39.823776   

    longitude  
0   -74.08323  
1  -74.416894  
2  -74.075484  
3  -74.249293  
4  -74.259258  


In [12]:
#extract USD value from dictionary
region_list["USD($)"] = [d.get('#text') for d in region_list.zindex]

region_list

AttributeError: 'DataFrame' object has no attribute 'zindex'

In [11]:
region_list[['Counties','type']] = region_list['name'].str.split('C',expand=True)
region_list

KeyError: 'name'

In [10]:
#Clean up County Column for FEMA


region_list = region_list["name"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
region_list["Counties"]= region_list[0] 
  
# making separate last name column from new data frame 
region_list["type"]= region_list[1] 
  
# Dropping old Name columns 
region_list.drop(columns =["name"], inplace = True) 
  
# df display 

region_list

KeyError: 'name'

In [ ]:
region_list["USD($)"] = pd.to_numeric(region_list["USD($)"])

region_list.dtypes

In [ ]:
#merge fema and zillow
fema_zillow_mrg = pd.merge(left=region_list,right=county_dis, left_on='Counties', right_on='Counties')
fema_zillow_mrg["latitude"] = fema_zillow_mrg["latitude"].astype(float)
fema_zillow_mrg["longitude"] = fema_zillow_mrg["longitude"].astype(float)
fema_zillow_mrg.head(30)

In [ ]:
plt.figure(figsize=(20,3))
plt.bar(fema_zillow_mrg["Counties"], fema_zillow_mrg["Disaster"], color='r', alpha=0.5, align="center")
plt.xticks(fema_zillow_mrg["Counties"], rotation="vertical")
#plt.xticks(rotation=90)

In [ ]:
multi_plot = region_list.plot(kind="bar", figsize=(20,5))

# PandasPlot.set_xticklabels() can be used to set the tick labels as well
multi_plot.set_xticklabels(fema_zillow_mrg["Counties"], rotation=45)
multi_plot.set_xlabel("Counties")
multi_plot.set_ylabel("Median House Prices($)")
multi_plot.set_title("Texas Median House Prices by County")
plt.show()
plt.tight_layout()

In [ ]:
#Heat Map of Disaster
locations = fema_zillow_mrg[["latitude", "longitude"]]
weight1 = fema_zillow_mrg["Disaster"]

fig = gmaps.figure(map_type="TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=weight1)
                                #dissipating=False, max_intensity=100)#, point_radius=1)

fig.add_layer(heat_layer)

fig

In [ ]:
# Map of Median House Price vs Disasters in State of Texas

info_box_template = """
<dl>
<dt>Name</dt><dd>{Counties}</dd>
<dt>HomePrice</dt><dd>{USD($)}</dd>
<dt>Disaster</dt><dd>{Disaster}</dd>
</dl>
"""
# Store the DataFrame Row
tx_info = [info_box_template.format(**row) for index, row in fema_zillow_mrg.iterrows()]
locations = fema_zillow_mrg[["latitude", "longitude"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f"County: {rate}" for rate in tx_info])
fig.add_layer(markers)
# Display Map
fig

# Display Map